In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error opening video camera")
    exit()

# BG Subtractor
bgSub = cv2.createBackgroundSubtractorMOG2()

prev_fingertip_center = None
is_tapping = False
tap_start_frame = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if frame was read successfully
    if not ret:
        print("Error reading frame!")
        break

    keyboard_layout = {
        (0, 0): 'A',
        (0, 1): 'S',
        (0, 2): 'D',
        (0, 3): 'F',
    }

    keyboard_width = 300
    keyboard_height = 150

    keyboard_overlay = np.zeros((keyboard_height, keyboard_width, 3), dtype="uint8")
    keyboard_overlay.fill(0)
    keyboard_overlay = cv2.resize(keyboard_overlay, dsize=(frame.shape[1], frame.shape[0]))

    for (row, col), key in keyboard_layout.items():
        x = int(keyboard_width / 2 * col)
        y = int(keyboard_height / 2 * row)

        key_width = int(keyboard_width / 4)
        key_height = int(keyboard_height / 2)

        cv2.rectangle(keyboard_overlay, (x, y), (x + key_width, y + key_height), (255, 255, 255), 2)

        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        font_thickness = 2
        text_size, _ = cv2.getTextSize(key, font, font_scale, font_thickness)
        text_x = int((x + key_width / 2) - (text_size[0] / 2))
        text_y = int((y + key_height / 2) - (text_size[1] / 2))
        cv2.putText(keyboard_overlay, key, (text_x, text_y), font, font_scale, (255, 255, 255), font_thickness)

    # Apply background subtraction
    fgmask = bgSub.apply(frame)

    contours, _ = cv2.findContours(fgmask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Initialize variables for fingertip properties
    fingertip_contour = None
    max_area = 0

    # Loop through contours and analyze properties
    for cnt in contours:
        # Calculate area and convexity defects
        area = cv2.contourArea(cnt)
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        hull_defect = hull_area - area

        # Analyze potential fingertip based on area, convexity, and aspect ratio
        aspect_ratio = float(cv2.boundingRect(cnt)[2]) / cv2.boundingRect(cnt)[3]  # Width / Height

        # Update fingertip_contour if a better candidate is found
        if area < 1000 and hull_defect > 50 and aspect_ratio > 1:  # Adjust thresholds as needed
            fingertip_contour = cnt
            max_area = area

    # Draw fingertip contour (if detected)
    if fingertip_contour is not None:
        cv2.drawContours(frame, [fingertip_contour], -1, (0, 255, 0), 2)

    # Display the frame with foreground mask (optional)
    alpha = 0.7
    beta = 1 - alpha
    frame_with_overlay = cv2.addWeighted(frame, alpha, keyboard_overlay, beta, 0)
    cv2.imshow('Frame', frame_with_overlay)

    # Wait for a 'q' key press to quit
    if cv2.waitKey(1) == ord('q'):
        break

# Release the capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


2024-05-17 08:16:57.673 Python[56248:1760359] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
